In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)
from utils.downloads import _prepare_cache

# Intersecção entre setores censitários (2022) e a área de 1km a partir de Parques Urbanos e Lineares (buff)

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

Eu interpretei esse título como <=1km, não como <1km

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende dos arquivos resultantes dos notebooks '.buffed_parques_urb_line' e '../load_data/setores_censitario'

In [5]:
gdf_setores22= gpd.read_parquet(
    os.path.join(
        assets_path,
        'setores_2022.parquet'
    )
)

In [6]:
gdf_buffed_parq = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'area_1km_parques_urbanos_lineares.parquet'        
    )
)

# Overlay setores por buff

In [7]:
gdf_setores22['area_setor']=gdf_setores22['geometry'].area

In [8]:
overlay_buff_setores= gpd.overlay(
    gdf_setores22,
    gdf_buffed_parq,
    how='intersection',
    keep_geom_type=True    
)

# Conferir 

! fazer conferência das geometrias etc

# Calcular a população

aqui eu explicarei a metodologia

In [9]:
overlay_buff_setores['area_recort']=overlay_buff_setores['geometry'].area

In [10]:
overlay_buff_setores['percent_area'] = overlay_buff_setores['area_recort']/overlay_buff_setores['area_setor']

In [11]:
overlay_buff_setores['pop_recort']=round(overlay_buff_setores['total_pop']*overlay_buff_setores['percent_area'], 0)

In [12]:
overlay_buff_setores.sample(2)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,cd_parq_mun,geometry,area_recort,percent_area,pop_recort
25433,355030867000123,Urbana,1,0,3550308,São Paulo,355030867,Rio Pequeno,35503086700,None,...,3550308,São Paulo/SP,442.0,170.0,36742.772269,34,"POLYGON ((321149.664 7391853.468, 321129.075 7...",17773.797015,0.483736,214.0
19254,355030847000187,Urbana,1,0,3550308,São Paulo,355030847,José Bonifácio,35503084700,None,...,3550308,São Paulo/SP,579.0,224.0,25759.044035,101,"POLYGON ((355141.572 7391280.016, 355149.914 7...",25759.044035,1.000000,579.0


# Salvar gdf

In [13]:
save_parquet_excel(
    overlay_buff_setores,
    fname='overlay_setores22_buff_parques_urb_line',
    data_path = assets_path,
    data_subpath = 'parques_lineares_e_urbanos'
)